# Modeling

## Import data

In [1]:
import pandas as pd

# Read dataframe from the pickle format
wide_df = pd.read_pickle("../../data/processed/preprocessed_df.pkl")

## Select features

In [2]:
X = wide_df[["sex_gender", "age", "eeg", "engel_outcomes", "mri",
             "neuroanatomical_labeling", "neuropsychological_testing", 
             "surgical_information"]]
y = wide_df["surg_engel"]

## Split data into train/test

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

## Scale continuous variables

In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

column_transformer = ColumnTransformer(
    transformers=[
        ('age', StandardScaler(), ['age'])
        # Add more transformers for other columns if needed
    ],
    remainder='passthrough'  # Keep the columns not specified in transformers unchanged
)

X_train_scaled = column_transformer.fit_transform(X_train)
X_test_scaled = column_transformer.transform(X_test)

## Decision Tree classifier

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

clf = DecisionTreeClassifier(random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.59


## MODEL???